# Exercises

In [1]:
import numpy as np
import pandas as pd

## Exercise 1

What is the approximate depth of a decision tree trained (without restrictions) on a training set with one million instances?

---

No restrictions means that each leaf will have with 1 training instance. On average the tree will be balanced, so the number of children will double whenever we increase the depth. The question is how long can this continue until we run out of training instances. The answer is $log_2(m)$

In [3]:
print(np.log2(1e6))

19.931568569324174


So the depth will be 20 on average.

## Exercise 2

Is a node's Gini impurity generally lower or greater than its parent's? It is *generally* lower/greater, or *always* lower/greater?

---

The Gini impurity of a child is generally lower than its parent's because when splitting nodes we seek to minimise the weighted impurity of the children:

\begin{equation}
    \frac{m_{\text{left}}}{m}G_{\text{left}} + \frac{m_{\text{right}}}{m}G_{\text{right}}.
\end{equation}

However, one child may have higher impurity if this is compensated for in the weighted sum by a lower impurity in the other child.

## Exercise 3

If a decision tree is overfitting the training set, is it a good idea to try decreasing `max_depth`?

---

Yes. Decreasing `max_depth` simplifies the model and hence decreases the chance of overfitting.

## Exercise 4

If a decision tree is underfitting the training set, is it a good idea to try scaling the input features?

----

No. The decision tree training algorithm is scale invarient, so this won't make any difference.

## Exercise 5

If it takes one hour to train a decision tree on a training set contained 1 million instances, roughly how much time will it take to train another decision tree on a training set containing 10 million instances.

---

The computational complexity of training a decision tree is $O(m \log_2(m))$. So the increase in complexity will be:

In [8]:
cplx_old = 1e6 * np.log2(1e6)
cplx_new = 1e7 * np.log2(1e7)
cplx_new / cplx_old

11.666666666666666

So training the new tree will take approximately 11-12 hours.

## Exercise 6

If your training set contains 100,000 instances, will setting `presort=True` speed up training?

---

No. With a training set that size, presorting will probably take more time than it will actually save in traning.